<img src="./images/DLI_Header.png" width=400/>

# Reconstructing Outdoor Environments for Physical AI Simulation with 3D Gaussian Splatting in NVIDIA Isaac Sim

**Table of Contents**
<br>
In this notebook we demonstrate how to reconstruct an outdoor scene. This includes the following sections:

[Part 1: Creating a Digital Twin with Gaussian Splatting](#part-1-creating-a-digital-twin-with-gaussian-splatting)<br>
    &emsp;&emsp;1. [Gather Images](#gather-images)<br>
    &emsp;&emsp;2. [Run Structure from Motion (COLMAP)](#run-structure-from-motion-colmap)<br>
    &emsp;&emsp;3. [fVDB Reality Capture](#fvdb-reality-capture)<br>
    &emsp;&emsp;&emsp;&emsp;a) [Viewing a Sfm Scene](#viewing-a-sfm-scene) <br>
    &emsp;&emsp;&emsp;&emsp;b) [Reconstructing a Scene with Gaussian Splatting](#reconstructing-a-scene-with-gaussian-splatting) <br>
    &emsp;&emsp;4. [Editing Gaussian Splats](#editing-gaussian-splats)<br>
    &emsp;&emsp;5. [Create Isaac Sim Ready Files](#create-isaac-sim-ready-files)<br>
    &emsp;&emsp;&emsp;&emsp;a) [Convert to a Mesh](#convert-to-a-mesh) <br>
    &emsp;&emsp;&emsp;&emsp;b) [Cropping and Converting](#cropping-and-converting) <br>
[Part 2: Creating an Isaac Sim Scene](#Part-2-creating-an-isaac-sim-scene)<br>
    &emsp;&emsp;6. [Running Isaac Sim](#running-isaac-sim)<br>
    &emsp;&emsp;7. [Import the Assets](#import-the-assets)<br>
    &emsp;&emsp;8. [Scene Setup](#scene-setup)<br>
    &emsp;&emsp;9. [Optional: Splat Color Editing](#optional-splat-color-editing)<br>
    &emsp;&emsp;10. [Save Scene](#save-scene)<br>
    &emsp;&emsp;11. [Isaac Lab and Robot Locomotion](#isaac-lab-and-robot-locomotion)

## Overview
In this lab we will learn to how to reconstruct an outdoor scene to test robots using NVIDIA fVDB framework and NVIDIA Omniverse NuRec rendering in Isaac Sim. We will walk through core reconstruction and rendering technologies, with a step-by-step workflow for simulating an entire outdoor environment for testing any robot. We will learn how to position images using structure from motion, train a 3D Gaussian splatting scene, extract 3D mesh, and convert to USD for simulation in Isaac Sim. This lab is split into two parts. In [Part 1](#part-1-creating-a-digital-twin-with-gaussian-splatting), we will learn how to gather good data, use a SfM tool, and train a Gaussian splatting scene. In [Part 2](#Part-2-creating-an-isaac-sim-scene), we will switch focus to Isaac Sim. We will import the files from [Part 1](#part-1-creating-a-digital-twin-with-gaussian-splatting) into Isaac Sim and use Isaac Lab to move a Spot robot around the scene.

## Part 1: Reconstructing a Scene with Gaussian Splatting

### Gather Images 
Your reconstruction can only be as good as the data you start with, because of this it's crucial to have good images. For outdoor scene there are some best practices you can follow to get good results. 
* If you have one main object or area you're focusing on it is recommend to circle or orbit it with a camera equipped aerial vehicle. More details on object centric collection can be found in [nerf_dataset_tips.md]( https://github.com/NVlabs/instant-ngp/blob/master/docs/nerf_dataset_tips.md).
* For larger outdoor scenes with no one specific focus either overlapping circular orbits, or traditional oblique mapping flight lines produce good quality results. An example flight mode for a popular drone model can be found [here](https://enterprise-insights.dji.com/blog/smart-oblique-capture).

In this lab, we will show an example of an orbit collection using a video captured during flight. Frames are extracted every second or so for training. [FFmpeg](https://www.ffmpeg.org/), an open-source software for audio and video file processing, can be used to save individual frames from a video. Below is an example of how you can save 1 frame per second out as an image from an input video.

<table style="width:80%;">
  <tr>
    <td style="width:50%; text-align:center;">
      <img src="images/safety_park_camera_poses.png" alt="First Image" style="width:100%;">
      <div>Camera positions for a scene with a single area of focus. Cameras point towards the center of the scene wile orbiting around it. This scene is Safety Park, we will be using this scene for the rest of the lab.</div>
    </td>
    <td style="width:50%; text-align:center;">
      <img src="images/civil_air_patrol_camera_poses.png" alt="Second Image" style="width:100%;">
      <div>Camera positions for a large scene with no specific single area of focus. In this case the camera makes many orbits, covering about 7 square kilometers.</div>
    </td>
  </tr>
</table>

In [ ]:
# save 1 frame per second from the original video to the **images_raw** folder as images.
!ffmpeg -i ../../Data/safety_park/safety_park.webm -vf fps=1 ../../Results/safety_park/images_raw/output_frame_%04d.png

For this lab we have curated data for you to use. We will be reconstructing Safety Park, a small pseudo town used for first responder training. Let's view the original video and some of the images of this park that we will use for training.

In [ ]:
# play video
! vlc ../../Data/safety_park/safety_park.webm



In [ ]:
# View images
from IPython.display import Image
Image(filename="../../Data/safety_park/images_raw/000059.jpg")

In [ ]:
Image(filename="../../Data/safety_park/images_raw/000114.jpg")

### Run Structure from Motion (COLMAP)
Many radiance field rendering methods, including Gaussian Splatting, require the camera positions and a sparse point cloud of the scene for initialization. We currently have a folder of raw images, but no corresponding camera location or pose information. We can use a structure from motion tool (SfM) to estimate where the camera was for each image and to create a sparse point cloud of the scene. A commonly used SfM tool is [COLMAP](https://colmap.github.io/install.html), which will use in combinations with GLOMAP. [GLOMAP](https://github.com/colmap/glomap) replaces COLMAP's mapper step, focusing on global positioning rather than incremental, and can run 10 or even 100 times faster than COLMAP's. Below we have provided example commands, there's no need to run them here as we have provided the result from an existing SfM run. You can run these commands on your own data by downloading COLMAP and GLOMAP and changing the **/Data/Path** to a directory that contains the **images_raw** folder of your data.


In [ ]:
# Example COLMAP & GLOMAP commands

# Run the feature extract to identify key points
!colmap feature_extractor \
    --database_path /Data/Path/database.db \
    --image_path /Data/Path/images_raw \
    --ImageReader.camera_model PINHOLE \
    --ImageReader.single_camera 1 \
    --SiftExtraction.use_gpu 1 \
    --SiftExtraction.max_image_size 7096 \
    --SiftExtraction.max_num_features 20000 \
    --SiftExtraction.num_threads 14

# Mature the features across images
!colmap exhaustive_matcher \
    --database_path /Data/Path/database.db \
    --SiftMatching.use_gpu 1 \
    --SiftMatching.max_num_matches 60000 \
    --SiftMatching.guided_matching=true

# Create a sparse 3D point cloud of the scene using GLOMAP's global mapper
!glomap mapper \
    --database_path /Data/Path/database.db \
    --image_path /Data/Path/images_raw \
    --output_path /Data/Path/sparse \
    --GlobalPositioning.use_gpu 1 \
    --BundleAdjustment.use_gpu 1

# Align 3d model by applying transformations
!colmap model_aligner \
    --input_path /Data/Path/sparse/0 \
    --output_path /Data/Path/sparse/aligned \
    --database_path /Data/Path/database.db \
    --ref_is_gps 1 \
    --alignment_type ECEF \
    --alignment_max_error 3.0

# Undistort original input images so they are as if they were taken with a pinhole camera
!colmap image_undistorter \
    --image_path /Data/Path/images_raw \
    --input_path /Data/Path/sparse/0 \
    --output_path /Data/Path \
    --output_type=COLMAP

Let's take a closer look at the files from the provided SfM run.

In [ ]:
# Print contents of directory in tree like format
!tree ../../Data/safety_park

Now let's delve deeper into each of these files and folders.

**images_raw**: Contains the original images <br>
**images**: Contains undistorted images <br>
**sparse**: Contains the sparse 3D reconstruction of the scene in folder 0. If COLMAP cannot register the images into 1 single scene, it will be split in to additional numbered folders. <br>
**cameras.bin**: Camera intrinsics including camera IDs, camera models, and sensor dimensions. <br>
**images.bin**: Camera poses and keypoints for all reconstructed images. <br>
**points3D.bin**: The sparse 3D point cloud <br>
To learn more these files COLMAP produces see the [COLMAP's Output Format page](https://colmap.github.io/format.html).


### fVDB Reality Capture
Now that we have a SfM run we can visualize it and use it to train a Gaussian splatting scene. We will use an [fVDB](https://github.com/openvdb/fvdb-core) example project called [fVDB Reality Capture](https://github.com/openvdb/fvdb-reality-capture) for visualization, manipulation and training. fVDB is a framework for encoding and operating on sparse voxel hierarchies of features in PyTorch. Voxels are like pixels but they are cubes instead of squares, making them three dimensional. Sparse means we only have voxels in areas of our scene that that are occupied, voxels that don't contain anything are not represented. fVDB Reality Capture (fRC) is toolbox for reality capture tasks built on top of ƒVDB. It gathers the tools required to create an Isaac Sim compatible 3D reconstruction from a set of images and uses fVDB to make the tools fast and efficient.



#### Setup and imports

Before we get started, let's import the pacakges we need. Here's an overview of some of the important ones:

* `logging`
    - We'll use the built-in Python `logging` module, and call `logging.basicConfig()` which will cause functions within `fvdb_reality_capture` to log to stdout. You don't have to enable this, but it's useful to see what's happening under the hood.
* `fvdb` 
    - We use `fvdb` for the underlying Gaussian splat data structure (`fvdb.GaussianSplat3d`) which provides fast and scalable rendering, and for interactive visualization (using the `fvdb.viz`) module.
* `fvdb_reality_capture` 
    - We use this for core algorithms that reconstruct scenes from sensors help us read and process capture data (Duh!)


In [ ]:
import logging

import cv2
import fvdb
import matplotlib.pyplot as plt
import numpy as np
import torch

import fvdb_reality_capture as frc

# Let's use verbose logging to track what happens under the hood.
# For less output set level=logging.WARN. For more set level=logging.DEBUG
logging.basicConfig(level=logging.INFO, format="%(levelname)s : %(message)s")

# Initialize the fvdb.viz module for interactive 3D visualization.
# This will spin up a small HTTP server in the background.
fvdb.viz.init(port=8080)

#### Displaying a Sfm Scene

We can use fRC to view and manipulate our SfM scene since it supports loading in capture data stored in different formats into a common representation that can be easily manipulated by users. To do this, data from a capture is stored in an `fvdb_reality_capture.SfmScene` object which acts as an in memory representation of a 3D capture. We will use this object to manipulate and visualize our SfM scene. Let's import the required libraries and load our scene into a `SfmScene`.

In [ ]:
# Load Safety Park SfM Scene
sfm_scene: frc.SfmScene = frc.SfmScene.from_colmap("../../Data/safety_park")
print("Loaded SfM Scene")


Let's take a closer look at what a SfmScene scene consists of.
* `SfmScene.cameras`: A dictionary mapping unique camera IDs to SfmCameraMetadata objects which camera parameters (e.g. projection matrices, distortion parameters). The size of this dictionary matches the number of cameras used to capture the scene (so if you scanned a scene with a pair of stereo cameras, then len(SfmScene.cameras) will be 2).
* `SfmScene.images`: A list of SfmImageMetadata objects which contain paths to the images and optional masks, a reference to the camera (SfmCameraMetadata) used to capture each image, their camera-to-world (and inverse) transformations, and the set of 3D points visible in each image.
* `points/points_rgb/points_err`: Numpy arrays of shape (N,3)/(N,3)/(N,) encoding known surface points in the scene, their RGB colors, and an unnormalized confidence value of the accuracy of that point. Note, N denotes the number of points here. <br>
Now that we have a loaded SfmScene, let's plot some of its images, and the projected 3D points within those images.

We can use this object to to display our training images and project the 3D points from the sparse point cloud on to them.  We'll visualize the 3D points and cameras interactively using `fvdb.viz` and we'll plot some images with their visible points using `matplotlib`. The `fvdb.viz` module provides a high performance debug visualizer written in [vulkan](https://www.vulkan.org/). It spins up a small HTTP server which streams a visualization to a notebook or a browser.

In [ ]:
# Visualize an image in an SfmScene and the 3D points visible from that images
# projected onto the image plane as blue dots.
def plot_image_from_scene(scene: frc.sfm_scene.SfmScene, image_id: int):
    image_meta: frc.sfm_scene.SfmPosedImageMetadata = scene.images[image_id]
    camera_meta: frc.sfm_scene.SfmCameraMetadata = image_meta.camera_metadata

    # Get the visible 3d points for this image
    visible_points_3d: np.ndarray = scene.points[image_meta.point_indices]

    # Project those points onto the image plane
    # 1. Get the world -> camera space transform and projection matrix
    world_to_cam_matrix: np.ndarray = image_meta.world_to_camera_matrix
    projection_matrix: np.ndarray = camera_meta.projection_matrix
    # 2. Transform world points to camera space
    visible_points_3d_cam_space = world_to_cam_matrix[:3,:3] @ visible_points_3d.T + world_to_cam_matrix[:3,3:4]
    # 3. Transform camera space coordinates to image space
    visible_points_2d = projection_matrix @ visible_points_3d_cam_space
    visible_points_2d /= visible_points_2d[2]

    # Load the image and convert to RGB (OpenCV uses BGR by default)
    loaded_image = cv2.imread(image_meta.image_path)
    assert loaded_image is not None, f"Failed to load image at {image_meta.image_path}"
    loaded_image = cv2.cvtColor(loaded_image, cv2.COLOR_BGR2RGB)

    # Plot the image and projected points
    plt.title(f"SfmScene Image {image_id}")
    plt.axis("off")
    plt.imshow(loaded_image)
    plt.scatter(visible_points_2d[0], visible_points_2d[1], color="#432de9", marker=".", s=2)

# Visualize the SfmScene interactively in a 3D viewer using fvdb.viz.Viewer
def visualize_sfm_scene(scene: frc.sfm_scene.SfmScene,
                        name: str,
                        center_scene: bool = False):

    viewer_scene = fvdb.viz.get_scene("SfmScene Visualization")
    # Optionally center the scene at the origin.
    # This is useful to visualize multiple scenes together without them being far apart.
    if center_scene:
        center_transform = np.eye(4)
        center_transform[:3, 3] = -np.median(scene.points, axis=0)
        scene = scene.apply_transformation_matrix(center_transform)

    # Plot the points in the SfmScene with their colors (which are uint8 by default but the viewer
    # expects float32 colors in [0,1]).
    # Each point is drawn as a small sphere with a 2 pixel radius.
    viewer_scene.add_point_cloud(
        name=f"{name} Points",
        points=scene.points,
        colors=scene.points_rgb.astype(np.float32) / 255.0,
        point_size=2.0)

    # Plot the cameras as coordinate frames with axis length 2 units,
    # and frustums whose distance from the origin to camera plane is 1 unit long.
    viewer_scene.add_cameras(
        f"{name} Cameras",
        camera_to_world_matrices=scene.camera_to_world_matrices,
        projection_matrices=scene.projection_matrices,
        axis_length=2,
        frustum_scale=2.5,
    )

    # Set the initial camera view to be at the position of the first posed image, in the SfmScene,
    # looking at the center of the 3D points, with Z as up (COLMAP SfM scenes use Z as up).
    viewer_scene.set_camera_lookat(
        eye=scene.image_camera_positions[0],
        center=np.zeros(3),
        up=np.array([0, 0, 1]),  # Z is up in COLMAP SfM scenes
    )


# Plot three images from the scene and their visible 3D points alongside each other
plt.figure(figsize=(25, 4.25))
plt.subplot(1, 3, 1)
plot_image_from_scene(sfm_scene, 8)
plt.subplot(1, 3, 2)
plot_image_from_scene(sfm_scene, 16)
plt.subplot(1, 3, 3)
plot_image_from_scene(sfm_scene, 32)
plt.show()

# View the SfmScene interactively in a 3D viewer
visualize_sfm_scene(sfm_scene, "Raw SfmScene", center_scene=True)
fvdb.viz.show()

If you want to know more about manipulation of SfmScenes in fvdb see the [Reconstructing a Gaussian Splat Radiance Field and High Quality Mesh from a Capture
](https://github.com/openvdb/fvdb-reality-capture/blob/main/notebooks/radiance_field_and_mesh_reconstruction.ipynb) notebook.

#### Train a Gaussian Splatting Scene 
In this section we will walk through training a Gaussian splatting scene from a set of images and a SfM output, but let's start with what a Gaussian splatting scene is.

[3D Gaussian Splatting](https://repo-sam.inria.fr/fungraph/3d-gaussian-splatting/) is the dominant radiance field representation. A [radiance field](https://radiancefields.com/) is a 3D scene representation that specifies the color of a point along a given view direction. Radiance fields enable high fidelity visualization of 3D captures with realistic lighting effects. 3D Gaussian splats encode a radiance field function as a sum of 3D Gaussian functions or splats, parameterized by their means (positions), rotations, and scales. Additionally each splat has an opacity value and a set of spherical harmonics that map the direction in which a Gaussian is viewed to a color. Each scene is made up of thousands to millions of these 3-dimensional splats. Think of them as similar to voxels, or points in a point cloud. We start with splats in the same positions as the points in the sparse point cloud we got from our SfM run. During training splats are added, removed, split, and changed until the Gaussian splatting scene is capable of producing renders that are nearly identical to the training images. 

Now that we have an understanding of what a Gaussian splatting scene is, let's train our own. The core API for Gaussian Splat reconstruction is the `fvdb_reality_capture.GaussianSplatReconstruction` class. It accepts an input `SfmScene` and optional config parameters, and produces an `fvdb.GaussianSplat3d` reconstructing the scene. This functionally is also written in `train.py`. We will show how to use both methods.

In [ ]:
from fvdb_reality_capture.training import Config, SceneOptimizationRunner
# Create config so we can edit various parameters
cfg = Config();
cfg.eval_at_percent = [100]; # save evaluation for the end of training
cfg.save_at_percent = [100]; # Don't save the model until the end of training
cfg.refine_every_epoch = 4.5; # How often to refine Gaussians during optimization
cfg.pose_opt_start_epoch = 100; # Epoch at which we start optimizing camera positions

# Create a SceneOptimizationRunner
runner = frc.radiance_fields.GaussianSplatReconstruction.from_sfm_scene(
        config=cfg, # add the config with the parameters we changed
        dataset_path="../../Data/safety_park", # Path to data
        results_path="../../Results", # Where to save results
        image_downsample_factor=2, # Factor to scale images down by
        disable_viewer=True, # Turn off live viewer
        normalization_type="ecef2enu", # Normalization type, using Earth-centered Earth-fixed coordinates to local east-north-up
        points_percentile_filter= 2, # Amount of points that is too few for an image to contain and still use for training
        use_every_n_as_val= -1, # Don't leave any images for validation at the end, use all for training
    );

# Train
runner.optimize();


Higher resolution images and a larger quantity of images will cause training to be longer. We have pretrained splat for you to use. To stop the training, click the square button at the top of the notebook to interrupt the kernel.

Our optimizer was created with several different parameters. They effect when we save the model, how often the splats are refined, and more. There are many more parameters you can vary for training, run the training script with the help flag to see them all.

In [ ]:
!python train.py -h

Trying changing parameters to see how they effect training. We have added some for you to adjust and build off of. To stop the run early you can click the square button at the top of the notebook to interrupt the kernel.

In [ ]:
# Try changing and adding parameters
!python train.py --dataset-path ../../Data/safety_park --results_path ../../Results --image-downsample-factor 4 --cfg.max-epochs 100 --cfg.eval-at-percent 100 --disable_viewer

#### Visualize a Gaussian Splatting Scene
A major benefit of a 3D radiance fields is that we can render them continuously from any point in space in real time. Let's interactively visualize the reconstructed Gaussian Splat, examining the result from novel, freeform viewpoints. The viewer in `fvdb.viz` makes this straightforward by letting us visualize `fvdb.GaussianSplat3d` objects directly.

In [ ]:
# Get the trained splat model from the reconstruction runner
model: fvdb.GaussianSplat3d = runner.model
# model, metadata = fvdb.GaussianSplat3d.from_ply("./reconstructed_model.ply")

# Clear previous contents from the viewer
# viewer.clear()

# Add our splat model to the viewer
scene = fvdb.viz.get_scene("Gaussian Splat Model Visualization")
scene.add_gaussian_splat_3d("Reconstructed Gaussian Splat Radiance Field", model)

scene.add_cameras(
    "Input Cameras",
    camera_to_world_matrices=cleaned_sfm_scene.camera_to_world_matrices,
    projection_matrices=cleaned_sfm_scene.projection_matrices,
    axis_length=2,
    frustum_scale=2.5,
)

# Set up the viewer's initial camera to be positioned at the first camera in the SfmScene
# looking at the center of the scene. This should give a good initial view of the model.
camera_position = cleaned_sfm_scene.images[0].origin
camera_lookat_point = model.means.mean(dim=0)
scene.set_camera_lookat(eye=camera_position, center=camera_lookat_point, up=(0, 0, 1)) # Colmap uses Z as up
fvdb.viz.show()

### Editing Gaussian Splats
[Supersplat](https://github.com/playcanvas/supersplat) is an easy to use web browser based tool for viewing and editing Gaussian splatting scenes. We will use a local version, but in the future you can access it from [superspl.at/editor](https://superspl.at/editor) or locally. Let's launch this tool and start editing our splats. 


In [ ]:
!cd ../supersplat && npm run develop;

1. To see the viewer, go to [localhost:3000](http://localhost:3000/) in a web browser. 
2. In the upper left of the window navigate to to **File > Import**
3. In the pop up file explore navigate to **/home/nvidia/Reconstructing_Outdoor_Environments/Data/safety_park_output**, and import the **safety_park_splats.ply**.


In [ ]:
# Run this cell to see the Supersplat demo video
from IPython.display import Video
Video("./images/Supersplat_demo.webm", width=800, embed=True)

Now that the Gaussian splatting scene you created is displayed in Supersplat, you can move around, zoom in and out, and edit the splat. Experiment with the tools, such as the transform and selection tools. When your're done, shut down Supersplat by using the square button at the top of the notebook to interrupt the kernel.

### Create Isaac Sim Ready Files
The PLY file containing a Gaussian splat scene we get at the end of training is not compatible with Isaac. Let's go over how we can get our scene ready for use in Isaac Sim.

#### Convert to a Mesh
In order for a robot to move around in Isaac Sim there needs to be a surface it can interact with. Gaussian splatting scenes cannot represent a solid surface in Isaac Sim, so we need to convert our splats to something capable of acting as a surface a robot can collide with. Specifically we will convert the splats into a mesh. Our Gaussian splat model is capable of producing images and depths from novel views. A natural way to reconstruct a mesh from a Gaussian scene is by rendering depth maps from the camera poses used to reconstruct the model and fuse them into a truncated signed distance field (TSDF) using the TSDF fusion algorithm<sup>[\[2\]](#references)</sup>. 

TSDF fusion accumulates noisy depth maps into a voxel grid, to approximate a signed distance field near the surface of the object. `fvdb_reality_capture` makes use of `fvdb-core` to provide a high performance implementation of TSDF integration on sparse voxels. This allows us to generate meshes from Gaussian splats at very high resolutions on the GPU while using minimal memory.

The TSDF fusion algorithm is based on the paper: [KinectFusion: Real-Time Dense Surface Mapping and Tracking](https://www.microsoft.com/en-us/research/publication/kinectfusion-real-time-3d-reconstruction-and-interaction-using-a-moving-depth-camera/).
The DLNR foundation model is described in the paper: [High-frequency Stereo Matching Network](https://openaccess.thecvf.com/content/CVPR2023/papers/Zhao_High-Frequency_Stereo_Matching_Network_CVPR_2023_paper.pdf).
In short, this algorithm works by rendering stereo pairs of images from multiple views of the Gaussian splat model, and using DLNR to compute depth maps from these stereo pairs. The depth maps and images are then integrated into a sparse `fvdb.Grid` in a narrow band around the surface using a weighted averaging scheme. The algorithm returns this grid along with signed distance values and colors (or other features) at each voxel.
The mesh can then be extracted from the TSDF using the marching cubes algorithm implemented in `fvdb.marching_cubes.marching_cubes`.

In [ ]:

--ply_path ../../Data/safety_park_half/pretrained_splat.ply \

import point_cloud_utils as pcu

# The truncation margin determines the width of the narrow band around the surface in which we compute the TSDF.
# A larger margin will produce coarser voxels, while a smaller margin will produce finer voxels but may miss some surface details.
# Here we pick a truncation margin of 0.25 world units in our scene.
truncation_margin = 0.5

# This function returns a tensor of vertices, faces, and colors for the mesh.
# The vertices have shape (num_vertices, 3), the faces have shape (num_faces, 3),
# and the colors have shape (num_vertices, 3). The colors are in the range [0, 1].
v, f, c = mesh_from_splats(model, cleaned_sfm_scene.camera_to_world_matrices, cleaned_sfm_scene.projection_matrices, cleaned_sfm_scene.image_sizes, truncation_margin)

# Save the mesh as a PLY file for viewing in external tools using point_cloud_utils (https://fwilliams.info/point-cloud-utils/) [3]
pcu.save_mesh_vfc("../../Results/safety_park_isaac_files/safety_park_mesh.ply", v.cpu().numpy(), f.cpu().numpy(), c.cpu().numpy())

print(f"Reconstructed mesh with {v.shape[0]:,} vertices and {f.shape[0]:,} faces")

#### Cropping and Converting

As we saw in Supersplat, splats end up beyond the main focus of our scene. The training process tries to reconstruct the details in all the images, even details far in the horizon. This can lead to messing looking edges. You can remove the edges in Supersplat but we will crop both the mesh and PLY to focus on the center of our scene using **create_isaac_sim_ready_files.py**. Isaac Sim also uses different world axes coordinates than COLMAP, it assumes Z+ is up rather than -Y, so we will rotate the scene accordingly. Additionally, to ensure our mesh will act as a collider in Isaac Sim we will make it water tight. This will fill in holes in the mesh and smooth it out. While we are at it we will convert the mesh from a PLY to an OBJ, a format compatible with Isaac Sim. Isaac Sim doesn't understand a Gaussian splatting scene in PLY format either, so we will have to convert ours into something Isaac Sim can render. We will convert it to a Universal Scene Description Zip (USDZ), this is a compressed version of the USD file type commonly used with Isaac Sim.

In [ ]:
# Run the script to make Isaac Sim Ready Files
# --input_splat: Location of Gaussian splatting scene
# --input_mesh: Location of mesh (PLY)
# --output_path: Where to save the OBJ and USDZ (no file extension)
# --bbox: Box to crop to
# --resolution: How detailed our mesh will be. Increase if you want more faces and vertices.
!python scripts/create_isaac_ready_files.py \
--input-splat ../../Data/safety_park_half/pretrained_splat.ply \
--input-mesh ../../Data/safety_park_half/pretrained_mesh.ply \
--output-path ../../Results/safety_park_isaac_files/safety_park_cropped \
--bbox -100 -70 -20 110 90 20 \
--resolution 10000


## Part 2: Creating an Isaac Sim Scene

Isaac Sim 5.0 and above includes [NuRec (Neural Reconstruction) rendering](https://docs.isaacsim.omniverse.nvidia.com/5.0.0/assets/usd_assets_nurec.html), adding the functionality to render Gaussian splatting scenes among other neural volume methods. In this section we will go through the process of creating an environment from a Gaussian splatting scene and mesh. Then, using Isaac Lab, we will walk a [Spot](https://bostondynamics.com/products/spot/) robot around the scene. 

### Running Isaac Sim
To run Isaac Sim you need to locate the isaacsim folder, navigate to the release subdirectory, and run the isaac-sim.sh file. 

In [ ]:
!cd ../isaacsim/_build/linux-x86_64/release && ./isaac-sim.sh

After a few moments Isaac Sim will open and you should see the following window. 

<img src="./lab_images/Isaac_sim_launch.png" width="800" style="display:block; margin:auto;" />

### Import the Assets
Once Isaac Sim has launched you can import the Gaussian splatting scene and mesh made in [Part 1](#part-1-creating-a-digital-twin-with-gaussian-splatting).
1. In the content tab navigate to **/home/nvidia/Reconstructing_Outdoor_Environments/Data/isaac_files**, inside this folder is the **safety_park_mesh_res_50000.obj**, drag the file into the stage window on the right.
2. In the content window, still in **/home/nvidia/Reconstructing_Outdoor_Environments/Data/isaac_files**, drag **safety_park_splats.usdz** into the stage window.

In [ ]:
# Run this cell to see the Isaac Sim file import demo video
from IPython.display import Video
Video("./images/import_mesh_and_splats.webm", width=800, embed=True )

Now the 3D Gaussian scene and the mesh should be overlapping in the viewer. The mesh will be very small at first, let's see how we can fix that.

### Scene Setup
Let's setup our scene so its ready for a robot.
1. In the **Stage** tab click the **safety_park_mesh_res_50000** xform.
2. In the **Property** tab, under **Transform**, change the **Scale:unitsResolve** to 1.0 for X, Y and Z.
4. Back in the stage right click the **safety_park_mesh_res_50000** xform and select **Add > Physics > Colliders Preset**
    * This makes it so other objects collied with our mesh instead passing right through it.
5. Click the eye icon next to the **safety_park_mesh_res_50000** in the **Stage** window to hide the mesh.


In [ ]:
# Run this cell to see the align splats and mesh in Isaac Sim demo video
from IPython.display import Video
Video("./images/scene_setup.webm", width=800, embed=True )

Now the mesh and the 3D Gaussian scene are aligned. We also hid the mesh from view. It will still act as a collider but now we can use just high resolution splats for the visualization of Safety Park.

### Optional: Splat Color Editing
Currently, lights don't interact with the Gaussian splatting scene, just the mesh. Since changing the strength of lights in the environment will have no effect on the look of the splats, we can artificially change the lighting by changing the emissive color values of the Gaussian splatting scene.
1. In the **Stage** tab, select the **safety_park_splat > gauss > gauss > emissive_color_field** asset.
2. In the **Property** tab, under **Raw USD Properiteies**
    * Change Z in **emissive_color_field.omni:nurec:ccmB** to .7
    * Change Y in **emissive_color_field.omni:nurec:ccmG** to .7
    * Change X in **emissive_color_field.omni:nurec:ccmR** to .7

These values represent the amount the strength of emission for each of the 3 color channels. Be decreasing them all to 0.7 the splat looks less bright. Experiment with changing the values. Changing them non uniformly will result in changes to the color of the scene, rather than just the intensity.

In [ ]:
# Run this cell to see the ddjust splat colors in Isaac Sim demo video
from IPython.display import Video
Video("./images/color.webm", width=800, embed=True )

### Save Scene
To use the scene we created with Isaac Lab and a Spot robot we need to save it.
1. Navigate to **File > Save As...**
2. In the file browser that pops up navigate to **/home/nvidia/Reconstructing_Outdoor_Environments/Result/safety_park_isaac_files**
3. Save the scene as **isaac_sim_scene.usd**.
4. Exit out of Isaac Sim using the close button.

In [ ]:
# Run this cell to see the save scene as USD demo video
from IPython.display import Video
Video("./images/save_usd.webm", width=800, embed=True )

### Isaac Lab and Robot Locomotion
We can now use Isaac Lab and Isaac Sim to teleoperate a quadruped robot model (Boston Dynamics Spot) around our saved scene using a keyboard or a gaming controller. A locomotion policy was trained for this robot model using reinforcement learning in Isaac Lab, and we can inference this policy to translate velocity commands from the keyboard or controller into the joint-level actions required for the robot to walk. We need to launch Isaac Lab from outside our fVDB Python environment, so lets open a terminal.

1. Open a terminal

In [ ]:
!gnome-terminal --working-directory=/home/nvidia/Reconstructing_Outdoor_Environments/Code/IsaacLab

2. To ensure Isaac Lab has access to as much GPU as possible, lets kill the Jupyter Notebook. Go to **File > Shut Down**. When promoted confirm you want to shut down.

3. Isaac Lab needs to be ran outside of conda Python environments, including the fvdb environment and the base environment. Copy and paste the following command into the terminal so we are no longer working inside a conda environment. 

In [ ]:
# Deactivate conda environments
conda deactivate && conda deactivate && conda deactivate

4. Now we can open our scene with a locomotion policy equipped Spot robot using Isaac Lab. Copy and paste the following command into the terminal.

In [ ]:
# Use Isaac Lab to launch Isaac Sim with with a Spot robot and location policy in Isaac Sim
./isaaclab.sh -p /home/nvidia/Reconstructing_Outdoor_Environments/Code/robo_rl2/policy_inference_in_usd_safetypark.py --checkpoint /home/nvidia/Reconstructing_Outdoor_Environments/Code/robo_rl2/policy.pt --keyboard --terrain_usd /home/nvidia/Reconstructing_Outdoor_Environments/Data/isaac_files/safety_park_isaac_scene.usd

5. Use the arrow keys to move the Spot robot around. You can use the **X** and **Z** to control the yaw and turn the robot. Try running into objects to see how the collider is stopping the Spot.
6. The camera should follow the Spot robot around the scene as you explore Safety Park. You can change the position of the camera in the **Isaac Lab** tab on the right. Try changing the **Camera Eye** and **Camera Target** to see how it changes your view.

In [ ]:
# Run this cell to see the Isaac Lab demo video
from IPython.display import Video
Video("./images/isaac_lab.webm", width=800, embed=True )

Congratulations! You've completed this course, *Reconstructing Outdoor Environments for Physical AI Simulation with 3D Gaussian Splatting in NVIDIA Isaac Sim*. You can now create a 3D Gaussian Splatting reconstuction of a scene and use that scene with NVIDIA Omniverse in Isaac Sim and Isaac Lab.